In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_path = "/content/drive/MyDrive/mario_sentiment_新模型"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# 2. 定義情緒分析函式
def predict_with_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        score = probs[0][2].item() - probs[0][0].item()  # 正面 - 負面
        label = torch.argmax(probs, dim=1).item()
    label_map = {0: "負面", 1: "中立", 2: "正面"}
    return {
        "情緒分類": label_map[label],
        "情緒分數": round(score, 4)
    }

# 3. 分析整份檔案（逐行分析）
def analyze_file(df, text_column="text"):
    results = []
    for _, row in df.iterrows():
        analysis = predict_with_score(row[text_column])
        results.append({
            "句子": row[text_column],
            "情緒分類": analysis["情緒分類"],
            "情緒分數": analysis["情緒分數"]
        })

    # 總結
    scores = [r["情緒分數"] for r in results]
    avg_score = np.mean(scores)
    if avg_score > 1:
        overall = "結論為強烈正面"
    elif avg_score > 0:
        overall = "結論為正面"
    elif avg_score == 0:
        overall = "結論為中立"
    else:
        overall = "結論為負面"

    results.append({
        "句子": "【總結】",
        "情緒分類": overall,
        "情緒分數": round(avg_score, 4)
    })

    return pd.DataFrame(results)
# 匯入資料
df_input = pd.read_csv("/content/mario_comments_testset.csv")  # 確保有一欄是 "text"

# 執行分析
result_df = analyze_file(df_input, text_column="text")

# 匯出結果
result_df.to_csv("/content/情緒分析結果.csv", index=False, encoding="utf-8-sig")

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_path = "/content/drive/MyDrive/mario_sentiment_新模型"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# 2. 定義情緒分析函式
def predict_with_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        score = probs[0][2].item() - probs[0][0].item()  # 正面 - 負面
        label = torch.argmax(probs, dim=1).item()
    label_map = {0: "負面", 1: "中立", 2: "正面"}
    return {
        "情緒分類": label_map[label],
        "情緒分數": round(score, 4)
    }
def predict_sentiment_with_score(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        score = probs[0][1].item() - probs[0][0].item()  # 正面機率 - 負面機率

    # 分類規則：你可以根據需求調整
    if score > 1:
        sentiment = "強烈正面"
    elif score > 0:
        sentiment = "正面"
    elif score == 0:
        sentiment = "中立"
    else:
        sentiment = "負面"

    return {
        "情緒分類": sentiment,
        "情緒分數": round(score, 4),

    }
print(predict_sentiment_with_score("跟小朋友一起絕對是首推吧，小朋友選耀希就無敵了，加上，又沒時間限制可以慢慢通關"))
print(predict_sentiment_with_score("關卡都過不了，好好玩喔。"))
print(predict_sentiment_with_score("難玩至極"))

{'情緒分類': '負面', '情緒分數': -0.0125}
{'情緒分類': '負面', '情緒分數': -0.6162}
{'情緒分類': '負面', '情緒分數': -0.3281}
